<a href="https://www.kaggle.com/code/ocanaydin/football-cricket-basebal-transfer-learning?scriptVersionId=113934382" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**DIVIDE DATASET AS TRAIN AND VALIDATION.**

In [ ]:
base_dir = "../input/cricket-football-baseball/cricket-football-baseball"
classes = os.listdir(base_dir)
for i in range(len(classes)):
    path = os.path.join(base_dir,classes[i])
    print(f"{classes[i]} has {len(os.listdir(path))} images")

In [ ]:
import tensorflow as tf

In [ ]:
"""All images will be scaled 1/255 to normalize pixels between 0-1.Also image augmentation is used."""
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,rotation_range = 40,width_shift_range = 0.2,height_shift_range = 0.2,shear_range = 0.2,
    zoom_range = 0.2,horizontal_flip = True,fill_mode = "nearest" ,validation_split = 0.2
)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(base_dir,target_size = (300,300),class_mode = "categorical",
                                                                  batch_size = 4,subset = "training",seed = 42)
validation_generator = validation_datagen.flow_from_directory(base_dir,target_size = (300,300),class_mode = "categorical",
                                                         batch_size =4,subset = "validation",seed = 42)

**CNN ARCHITECTURE WITH TRANSFER LEARNING**

**(1)Use InceptionV3 as base model**

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (300,300,3),include_top = False,weights = "imagenet")
"""Freeze all layers to stop updating InceptionV3 trained weights."""
for layer in base_model.layers:
    layer.trainable = False


**(2)Create an architecture to feed model with images.**

In [ ]:
"""Here we can get the last layer as mixed8.It means that we can start updating our weights from mixed8 layer."""
"""You can change it if you want."""
last_layer = base_model.get_layer("mixed8")
print(last_layer.output_shape)

In [ ]:
"""Flatten layer to reduce output shape to 1dim."""
x = tf.keras.layers.Flatten()(last_layer.output)
"""Add fully connected layers with 256 units."""
x = tf.keras.layers.Dense(256,activation = "relu")(x)
"""Add Dropout layer."""
x = tf.keras.layers.Dropout(0.2)(x)
"""Output layer."""
x = tf.keras.layers.Dense(len(classes),activation = "softmax")(x)
"""Here we can connect our model end to end."""
model = tf.keras.models.Model(base_model.input,x)

In [ ]:
model.summary()

In [ ]:
model.summary()

**COMPILE AND FIT MODEL**

In [ ]:
model.compile(tf.keras.optimizers.RMSprop(learning_rate = 0.005),loss = "categorical_crossentropy",metrics = ["acc"])
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if logs.get("acc") > 0.95:
            print("Accuracy reached %95.Stop training.")
            self.model.stop_training = True 

callback = myCallback()

In [ ]:
history = model.fit(train_generator,epochs = 30,batch_size = 4,validation_data = validation_generator,verbose = 1,
                   callbacks = [callback])

**PLOT LOSS AND ACCURACY**

In [ ]:
import matplotlib.pyplot as plt
"""Plot accuracy"""
acc = history.history["acc"]
val_acc = history.history["val_acc"]
epochs = range(30)
plt.plot(epochs,acc,label = "Training Accuracy")
plt.plot(epochs,val_acc,label = "Validation Accuracy")
plt.legend()
plt.show()

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(30)
plt.plot(epochs,loss,label = "Training Loss")
plt.plot(epochs,val_loss,label = "Validation Loss")
plt.legend()
plt.show()

**SAVE MODEL AND LOAD MODEL**

In [ ]:
model.save("football_cricket_baseball_TFL.h5")

In [ ]:
model1 = tf.keras.models.load_model("football_cricket_baseball_TFL.h5")

**GET THE IMAGES FROM INTERNET AND PROCESS THEM**

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np

def get_and_process(url):
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img1 = img
        """Resize image for appropriate shape for model."""
        img = img.resize((300,300))
        """Convert img to numpy array,rescale it ,expand dims for model convenience. then check vertically."""
        x = tf.keras.preprocessing.image.img_to_array(img)
        x = x / 255.0
        x = np.expand_dims(x,axis = 0)
        img_tensor = np.vstack([x])
        return img1,img_tensor

**FINAL : PREDICT IMAGE**

In [ ]:
import matplotlib.pyplot as plt
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1d/Marcus_Thames_Tigers_2007.jpg/1200px-Marcus_Thames_Tigers_2007.jpg"
img1,test_img = get_and_process(url)
pred = model1.predict(test_img)
classes = list(train_generator.class_indices.keys())
print(f"Prediction is : {classes[np.argmax(pred)]}")
plt.imshow(img1)
plt.show()
